In [1]:
# V8 updates:
# 1 Clean up V7 for the web display

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import sampling0616 as sp
from matplotlib.patches import Rectangle

In [20]:
from ipywidgets import interact, interactive, fixed, interact_manual, HTML
import ipywidgets as widgets

In [21]:
import requests
from ipyleaflet import Map, basemaps, Marker, MarkerCluster, AntPath, FullScreenControl,\
                        LayersControl, WidgetControl, LayerGroup
import math
from functools import partial
import csv

## State of each location: calibrating the SOC and number of vehicles

In [5]:
class State:
    def __init__(self,max_soc=10):
        self.max_soc=max_soc
        self.veh=np.zeros(max_soc)
    
    def set_state(self,n,soc):
        try:
            self.veh[soc]=n
        except:
            print("SOC range error, soc in [0,max_soc-1]")
    def reset(self):
        self.veh-=self.veh
    
    def charging(self,charging_rate):
        if self.max_soc<2:
            return
        for i in range(self.max_soc-2,self.max_soc-charging_rate-2,-1):
            self.veh[self.max_soc-1]+=self.veh[i]
        for i in range(self.max_soc-charging_rate-2,-1,-1):
            self.veh[i+charging_rate]=self.veh[i]
        for i in range(charging_rate):
            self.veh[i]=0
    
    def moving(self,consumption_rate):
        if self.max_soc<2:
            return
        for i in range(1,consumption_rate+1):
            self.veh[0]+=self.veh[i]
        for i in range(consumption_rate+1,self.max_soc):
            self.veh[i-consumption_rate]=self.veh[i]
        for i in range(self.max_soc-consumption_rate,self.max_soc):
            self.veh[i]=0
            
    def add(self,o):
        self.veh+=o.veh
    def sub(self,o):
        self.veh-=o.veh
    def count(self):
        return sum(self.veh)
    
    def show(self):
        print("N[soc]:")
        print(self.veh)


## Each station and road segment is modeled as a node

In [6]:
class Node:
    def __init__(self,n_type,max_soc=10,location=(0,0),n_id='000',capacity_s=30,capacity_r=500):
        self.n_type=n_type
        self.state=State(max_soc)
        self.max_soc=max_soc
        self.loc=location
        self.clock=0
        self.sendbuffer=State(max_soc)
        self.receiveNodes=[]
        self.n_id=n_id
        if self.n_type==0:
            self.capacity =capacity_r
        if self.n_type==1:
            self.capacity =capacity_s
    
    def linking_node(self,nd):
        self.receiveNodes.append(nd)
        
    def dispatch(self,nd,demand): # this will be used for system dispatch, with no energy consumption
        self.state.veh-=demand
        nd.state.veh+=demand

    def send(self):
        #  station node       
        if self.n_type==1:
            return
        # road node
        self.sendbuffer.reset()
        self.sendbuffer.add(self.state)
        self.state.reset()
        
#     def receive(self):
#         if len(self.receiveNodes)==0:
#             return
#         for nd in self.receiveNodes:
#             self.state.add(nd.sendbuffer)  
    
    def receive(self,consumption_rate=0):
        if len(self.receiveNodes)==0:
            return
        
        # road node
        if self.n_type==0:
            for nd in self.receiveNodes:         
                self.state.add(nd.sendbuffer)
                
        # station node
        if self.n_type==1:
            for nd in self.receiveNodes:
                veh_list =nd.sendbuffer.veh
                arrival =np.zeros(self.max_soc)
                for i in range(0,consumption_rate+1):
                    arrival[0]+=veh_list[i]
                for i in range(consumption_rate+1,self.max_soc):
                    arrival[i-consumption_rate]=veh_list[i]
                for i in range(self.max_soc-consumption_rate,self.max_soc):
                    arrival[i]=0
                self.state.veh+=arrival
                    
    def update(self,consumption_rate,charging_rate):
        # road node
        if self.n_type==0:
            self.state.moving(consumption_rate)
            return
        
        #  station node 
        self.state.charging(charging_rate)

    def demand_num_to_list(self,distance,demand_num,consumption_rate):
        demand_array=np.zeros(self.max_soc)
        unsatisfied_demand=demand_num
        
        for i in range(self.max_soc-1,consumption_rate*distance-1,-1):
            if self.state.veh[i]>=unsatisfied_demand:
                demand_array[i]=unsatisfied_demand
                unsatisfied_demand=0
                break
            else:
                demand_array[i]=self.state.veh[i]
                unsatisfied_demand-=self.state.veh[i]
        return[demand_array,unsatisfied_demand]
        
    def data(self):
        return [self.clock,self.n_id,self.loc[0],self.loc[1]]+[int(self.state.veh[i]) for i in range(self.max_soc)]

    def show(self):        
        if self.n_type==1:
            plt.scatter([self.loc[0]],[self.loc[1]],s=100,color='r',marker='o')
        plt.scatter([self.loc[0]],[self.loc[1]],s=10*(self.state.count()+1),color='b')
        for nd in self.receiveNodes:
            plt.plot([nd.loc[0],self.loc[0]],[nd.loc[1],self.loc[1]],'b')
            

## Insert virtual road connecting two stations

In [7]:
def loc_inter(locA,locB,dist):
    inter_loc=[]
    vx=locB[0]-locA[0]
    vy=locB[1]-locA[1]
    x_shift=-0.1*vy/dist
    y_shift=0.1*vx/dist
#     y_shift=0
    for k in range(dist):
        r=(k+1)/(dist+1)
        x=(1-r)*locA[0]+r*locB[0]+x_shift
        y=(1-r)*locA[1]+r*locB[1]+y_shift
        inter_loc.append((x,y))
    return inter_loc

# A=(0,0)
# B=(1,1)
# road_loc=loc_inter(A,B,2)
# road_loc.append(A)
# road_loc.append(B)
# plt.plot([lc[0] for lc in road_loc],[lc[1] for lc in road_loc],'bo')
# plt.show()


## Create a network with multiple nodes and roads

In [8]:
class Network:
    def __init__(self,
                 location_map,
                 distance_matrix,
                 max_soc,
                 max_time,
                 consumption_rate,
                 charging_rate,
                 capacity_s,
                 capacity_r,
                 demand_m,
                 policy_para
                ):
        self.num_station=len(location_map)
        self.stations=[]
        self.roads={}
        self.distance={}
        self.max_soc=max_soc
        self.max_time=max_time
        self.consumption_rate=consumption_rate
        self.charging_rate=charging_rate
        self.capacity_r=capacity_r
        self.capacity_s=capacity_s
        self.demand=demand_m
        self.policy_para=policy_para
        self.dispatch_array={}
        for i in range(self.num_station):
            for j in range(self.num_station):
                for t in range(self.max_time):
                    for c in range(2): #1 system relocation or 0 customer demand
                        self.dispatch_array[i,j,t,c]=np.zeros(self.max_soc)
        
        table_head=['Time','NodeID','X','Y']+['L%d' % i for i in range(max_soc)]
        self.sim_result=pd.DataFrame(columns=table_head)
        
        for i in range(self.num_station):
            loc=location_map[i]
            n_id='t0s%d' % i
            self.stations.append(Node(1,max_soc,loc,n_id,self.capacity_s,self.capacity_r))
        for i in range(self.num_station):
            for j in range(self.num_station):
                if i==j:
                    continue
                self.distance[i,j]=distance_matrix[i,j]
                road_loc=loc_inter(location_map[i],location_map[j],self.distance[i,j])
                for k in range(self.distance[i,j]):
                    n_id='t1o%dd%dk%d' % (i,j,k)
                    self.roads[i,j,k]=Node(0,max_soc,road_loc[k],n_id,self.capacity_s,self.capacity_r)
                    if k>0:
                        self.roads[i,j,k].linking_node(self.roads[i,j,k-1])
                self.stations[j].linking_node(self.roads[i,j,self.distance[i,j]-1])

    
    def deploy_vehicle(self,veh_list):
        for i in range(self.num_station):
            self.stations[i].state.set_state(veh_list[i],self.max_soc-1)
        self._record() 
    
    def _record(self):
        # create dummy nodes for dispatch records
        # write the dispatch information for last time period. skip the period 0.
        t =self.stations[0].clock
        if t>=1:
            for i in range(len(self.stations)):
                for j in range(len(self.stations)):
                    if i==j:
                        continue
                    if t<=self.max_time:
                        #demand
                        demand_array=self.dispatch_array[i,j,t-1,0]
                        temp_data=[t-1,'t2o%dd%dc0'%(i,j),'-','-']+demand_array.tolist()
                        self.sim_result=self.sim_result.append(pd.Series(temp_data,index=self.sim_result.columns),ignore_index=True)
                        #relocation
                        relo_array=self.dispatch_array[i,j,t-1,1]
                        temp_data=[t-1,'t2o%dd%dc1'%(i,j),'-','-']+relo_array.tolist()
                        self.sim_result=self.sim_result.append(pd.Series(temp_data,index=self.sim_result.columns),ignore_index=True)
                        #total
                        total_array = self.dispatch_array[i,j,t-1,0]+self.dispatch_array[i,j,t-1,1]
                        temp_data=[t-1,'t2o%dd%d'%(i,j),'-','-']+total_array.tolist()
                        self.sim_result=self.sim_result.append(pd.Series(temp_data,index=self.sim_result.columns),ignore_index=True)
                        
        for nd in self.stations:
            self.sim_result=self.sim_result.append(pd.Series(nd.data(),index=self.sim_result.columns),ignore_index=True) 
        
        for key,nd in self.roads.items():
            self.sim_result=self.sim_result.append(pd.Series(nd.data(),index=self.sim_result.columns),ignore_index=True) 
    
    
    def show(self):
        for nd in self.stations:
            nd.show()
        for key,nd in self.roads.items():
            nd.show()
            
    def show_data(self):
        print(self.sim_result) 
        
    def show_frame(self,t):
        fig = plt.figure()
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        x=[]
        y=[]
        z=[]
        for i in range(len(dft)):
            xv=dft['X'].values[i]
            yv=dft['Y'].values[i]
            x.append(xv)
            y.append(yv)
            s=np.zeros(self.max_soc)
            for j in range(self.max_soc):
                s[j]=dft['L%d' % j].values[i]
            z.append(10*sum(s))
            plt.text(xv,yv,s)
        plt.scatter(x,y,s=z,color='b')
        plt.xlim([min(x),max(x)])
        plt.ylim([min(y),max(y)])
        
    def show_route(self,t,o,d):
        m=0.01
        edge=0.1
        
        fig = plt.figure()
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        x=[]
        y=[]
        z=[]
        dfto=dft.loc[dft['NodeID']=='t0s%d' % o]
        for h in range(self.max_soc):
            xv=0.0
            yv=(h+1)/self.max_soc
            zv=dfto['L%d' % h].values[0]
            x.append(xv)
            y.append(yv)
            z.append(zv)
            plt.text(xv-m,yv-m,zv)
            
        dftd=dft.loc[dft['NodeID']=='t0s%d' % d]
        for h in range(self.max_soc):
            xv=1.0
            yv=(h+1)/self.max_soc
            zv=dftd['L%d' % h].values[0]
            x.append(xv)
            y.append(yv)
            z.append(zv)
            plt.text(xv-m,yv-m,zv)
        
        for k in range(self.distance[o,d]):
            dftr=dft.loc[dft['NodeID']=='t1o%dd%dk%d' % (o,d,k)]
            for h in range(self.max_soc):
                xv=(k+1)/(self.distance[o,d]+1)
                yv=(h+1)/self.max_soc
                zv=dftr['L%d' % h].values[0]
                x.append(xv)
                y.append(yv)
                z.append(zv)
                plt.text(xv-m,yv-m,zv)
        plt.scatter(x,y,s=4200/self.max_soc*np.ones(len(x)),c=z,cmap='Wistia')
        plt.xticks(np.linspace(0,1,num=self.distance[o,d]+2), ['S%d'%o]+ \
                   ['K%d'% k for k in range(self.distance[o,d])] +['S%d'%d]) 
        plt.yticks(np.linspace(0,1,num=self.max_soc+1),['']+[ h for h in range(self.max_soc)])
        plt.xlabel('from station %d to station %d' % (o,d))
        plt.ylabel('SOC')
        plt.clim(0,self.capacity_s-1)
        
        # jump value ;arrow
        dftj=dft.loc[dft['NodeID']=='t2o%dd%d' % (o,d)]
        for h in range(self.max_soc):
            xv=0.3/(self.distance[o,d]+1)
            yv=(h+1)/self.max_soc
            zv=dftj['L%d' % h].values[0]
            if zv>=1:
                plt.text(xv,yv+0.01,int(zv))
                plt.arrow(xv,yv,0.3/(self.distance[o,d]+1),0,head_width=0.03,fc='k',ec='k')
        
        # dashed box
        ax = plt.gca()
        rect = Rectangle((min(x)-edge+0.03,min(y)-edge+0.03),1/(self.distance[o,d]+1)+2*edge-0.06,1,\
                         linewidth=1,edgecolor='k',facecolor='none',linestyle='dashed')
        ax.add_patch(rect)
        plt.xlim([min(x)-edge,max(x)+edge])
        plt.ylim([min(y)-edge,max(y)+edge])
        plt.colorbar()  
    
    def show_node(self,t,nd):
        m=0.02
        c1=500
        c2=50
        
        fig=plt.figure()
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        x=[]
        y=[]
        z=[]
        h=[] # ave battery
        # node nd
        xv=0.5
        yv=0.5
        xv_c=xv
        yv_c=yv
        dftnd=dft.loc[dft['NodeID']==nd.n_id]
        s=np.zeros(self.max_soc)
        for i in range(self.max_soc):
            s[i]=dftnd['L%d' % i].values[0]
        zv=c1+c2*sum(s)
        if sum(s)>0:
            hv=sum([h*s[h] for h in range(self.max_soc)])/sum(s)
        else:
            hv=0
        x.append(xv)
        y.append(yv)
        z.append(zv)
        h.append(hv)
        if nd.n_type==1:
            nd_index=int(nd.n_id[-1])
            plt.text(xv-m,yv-m,'S'+str(nd_index))
        if nd.n_type==0:
            plt.text(xv-m,yv-m,'O'+nd.n_id[-5]+'D'+nd.n_id[-3]+'K'+nd.n_id[-1])
        
        # incoming node
        in_num=len(nd.receiveNodes)
        ya=np.linspace(0,1,num=in_num+2)
        for i in range(in_num):
            xv=0.15
            yv=ya[i+1]
            dftin=dft.loc[dft['NodeID']==nd.receiveNodes[i].n_id]
            s=np.zeros(self.max_soc)
            for j in range(self.max_soc):
                s[j]=dftin['L%d' % j].values[0]
            zv=c1+c2*sum(s)
            if sum(s)>0:
                hv=sum([h*s[h] for h in range(self.max_soc)])/sum(s)
            else:
                hv=0
            x.append(xv)
            y.append(yv)
            z.append(zv)
            h.append(hv)
            plt.text(xv-2*m,yv-m,'S'+nd.receiveNodes[i].n_id[-5]+'K'+nd.receiveNodes[i].n_id[-1])
            plt.arrow(xv,yv,xv_c-xv-0.13,yv_c-yv-np.sign(yv_c-yv)*0.03,head_width=0.03,fc='b',ec='b')
            plt.text((xv+xv_c)/2,(yv+yv_c)/2+0.01,int(sum(s)),zorder=3)
        
        # outgoing node
        if nd.n_type==1:
            ya=np.linspace(0,1,num=self.num_station+1)
            j=0
            for i in range(self.num_station):
                if i==nd_index:
                    continue
                xv=0.85
                yv=ya[j+1]
                dftout=dft.loc[dft['NodeID']==self.stations[i].n_id]
                s=np.zeros(self.max_soc)
                for k in range(self.max_soc):
                    s[k]=dftout['L%d' % k].values[0]
                zv=c1+c2*sum(s)
                if sum(s)>0:
                    hv=sum([h*s[h] for h in range(self.max_soc)])/sum(s)
                else:
                    hv=0
                x.append(xv)
                y.append(yv)
                z.append(zv)
                h.append(hv)
                plt.text(xv-m,yv-m,'S'+str(i))
                plt.arrow(xv_c,yv_c,xv-xv_c-0.13,yv-yv_c-np.sign(yv-yv_c)*0.03,head_width=0.03,fc='b',ec='b')
                # demand 
                dftj0=dft.loc[dft['NodeID']=='t2o%dd%dc0' % (nd_index,i)]  
                s2=np.zeros(self.max_soc)
                for k in range(self.max_soc):
                    s2[k]=dftj0['L%d' % k].values[0]
                plt.text((xv+xv_c)/2,(yv+yv_c)/2+0.01,int(sum(s2)),zorder=3)
                # system dispatch
                dftj1=dft.loc[dft['NodeID']=='t2o%dd%dc1' % (nd_index,i)]
                s3=np.zeros(self.max_soc)
                for k in range(self.max_soc):
                    s3[k]=dftj1['L%d' % k].values[0]
                if sum(s3)>=1:
                    new_x,new_y=curve_between((xv_c,yv_c),(xv,yv))
                    plt.plot(new_x,new_y,'k',linestyle='dashed',zorder=1)
                    plt.text(new_x[round(len(new_x)/2)],new_y[round(len(new_y)/2)]-0.04,int(sum(s3)),zorder=3)
                j+=1
        # should work on this if decide to also plot the road node
        if nd.n_type==0:
            pass

        plt.scatter(x,y,s=z,c=h,cmap='cool',zorder=2)
        plt.clim(0,self.max_soc-1)
        plt.colorbar()
        plt.xlim([0,1])
        plt.ylim([0,1])
        plt.xticks([], [])
        plt.yticks([], [])
    
    def show_node_histogram(self,t,nd):
        fig=plt.figure()
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        dftnd=dft.loc[dft['NodeID']==nd.n_id]
        x=[]
        s=np.zeros(self.max_soc)
        for i in range(self.max_soc):
            s[i]=dftnd['L%d' % i].values[0]
            x+=[i]*int(s[i]) 
        plt.hist(np.array(x),bins=self.max_soc,range=[0,self.max_soc-1])
        tick_str=[]
        for i in range(self.max_soc):
            tick_str+=['',i]
        plt.xticks(np.linspace(0,self.max_soc-1,num=2*self.max_soc+1), tick_str+[''])
        plt.gca().xaxis.set_ticks_position('none')
        plt.xlabel('SOC')
        plt.ylabel('EV number')
    
    def show_map(self,t):
        c1=500
        c2=50
        
        fig=plt.figure()
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        dfts=dft[dft['NodeID'].str.contains('t0')]
        x=[]
        y=[]
        z=[]
        h=[]
        for i in range(len(dfts)):
            xv=dft['X'].values[i]
            yv=dft['Y'].values[i]
            x.append(xv)
            y.append(yv)
            s=np.zeros(self.max_soc)
            for j in range(self.max_soc):
                s[j]=dft['L%d' % j].values[i]
            z.append(c1+c2*sum(s))
            if sum(s)>0:
                hv=sum([h*s[h] for h in range(self.max_soc)])/sum(s)
            else:
                hv=0
            h.append(hv)
            plt.text(xv,yv,'S'+str(i)+':'+str(int(sum(s))))
        plt.scatter(x,y,s=z,c=h,cmap='cool')
        plt.clim(0,self.max_soc-1)
        plt.colorbar()
        edge=max(max(x)-min(x),max(y)-min(y))/10
        plt.xlim([min(x)-edge,max(x)+edge])
        plt.ylim([min(y)-edge,max(y)+edge])
        plt.xticks([], [])
        plt.yticks([], [])
        
    def _des_idle_capacity(self,des,dis):
        num_at_d =self.stations[des].state.count()
        for i in range(self.num_station):
            if i==des:
                continue
            if self.distance[i,des]>=dis:
                for k in range(self.distance[i,des]-1,self.distance[i,des]-1-dis,-1):
                    num_at_d+=self.roads[i,des,k].state.count()
            if self.distance[i,des]<dis:
                for k in range(self.distance[i,des]):
                    num_at_d+=self.roads[i,des,k].state.count()
        idle_capacity=self.stations[des].capacity-num_at_d
        return idle_capacity
    
    def _sys_dispatch_policy(self,policy_para,t):
        ev_num=[self.stations[i].state.count() for i in range(self.num_station)]
        ind_s = list(np.argsort(ev_num)) # rank of EV numbers of each station
        ind_s.reverse() # largest to smallest
        rank=1
        for i in ind_s:
            if ev_num[i] >=policy_para[0]*self.stations[i].capacity:
                des=ind_s[-rank]
                num_to_sent = min(policy_para[1]*self.stations[i].capacity, \
                                self.stations[des].capacity-self._des_idle_capacity(des,self.distance[i,des]))
                demand_array=self.stations[i].demand_num_to_list(self.distance[i,des],num_to_sent,self.consumption_rate)[0]
                self.stations[i].dispatch(self.roads[i,des,0],demand_array)
                self.dispatch_array[i,des,t,1]+=demand_array
                if self.congestion_staus(i,des)==0:
                    self._ori_1st_node_jump(i,des)
                rank+=1
    
    def _demand_able_to_fulfill(self,demand_num,i,j):
        unsatisfied_demand=0 # two types:1 destination full; 2 origin not enough EV       
        # check destination capacity
        idle_capacity =self._des_idle_capacity(j,self.distance[i,j])
        if demand_num>idle_capacity:
            unsatisfied_demand=demand_num-idle_capacity
            demand_num=idle_capacity
                    
        # turn demand into EV list
        [demand_array,unsatis_demand]=self.stations[i].demand_num_to_list(self.distance[i,j],demand_num,self.consumption_rate)
        unsatisfied_demand+=unsatis_demand
        return[demand_array,unsatisfied_demand]
                           
    def _ori_1st_node_jump(self,o,d):
        jump_ev=self.roads[o,d,0].state.veh
        for i in range(0,self.consumption_rate+1):
            jump_ev[0]+=jump_ev[i]
        for i in range(self.consumption_rate+1,self.max_soc):
            jump_ev[i-self.consumption_rate]=jump_ev[i]
        for i in range(self.max_soc-self.consumption_rate,self.max_soc):
            jump_ev[i]=0
        
        # jump
        self.roads[o,d,0].state.veh=np.zeros(self.roads[o,d,0].max_soc)       
        if self.distance[o,d]>=2:
            self.roads[o,d,1].state.veh+=jump_ev
        else:
            self.stations[d].state.veh+=jump_ev
        
    def congestion_staus(self,o,d):
        # 0 not conngested
        return 0 
    
    def update_time(self):
        for nd in self.stations:
            nd.clock+=1
        for key,nd in self.roads.items():
            nd.clock+=1
  
    def _action(self,policy_para,t):  
        # customer part
        for i in range(self.num_station):
            for j in range(self.num_station):
                if i==j:
                    continue
                demand_num = self.demand[i,j,t]
                [demand_array,unsatisfied_demand]=self._demand_able_to_fulfill(demand_num,i,j)   
                
                # push the rest demand to next time. [demand queue/node to enable further operation]
                if t<=self.max_time-2:
                    self.demand[i,j,t+1]+=unsatisfied_demand
                
                self.stations[i].dispatch(self.roads[i,j,0],demand_array)
                self.dispatch_array[i,j,t,0]+=demand_array
                if self.congestion_staus(i,j)==0:
                    self._ori_1st_node_jump(i,j)
        
        # system part
        self._sys_dispatch_policy(policy_para,t)
        
    def run(self):
        # discuss: station update(charge before receive)
        for t in range(self.max_time): 
            for key,nd in self.roads.items():
                nd.send()
            for key,nd in self.roads.items():
                nd.receive()     
            for key,nd in self.roads.items():
                nd.update(self.consumption_rate,self.charging_rate)
                                     
            for nd in self.stations:
                nd.send()  
            for nd in self.stations:
                nd.update(self.consumption_rate,self.charging_rate)
            for nd in self.stations:
                nd.receive(self.consumption_rate)
                  
            self._action(self.policy_para,t)
            self.update_time()
            self._record()
  

In [9]:
# curve between two points
def curve_between(p1,p2):
    # works for points with different x value
    a=2
    mid_x=(p1[0]+p2[0])/2
    x = np.linspace(p1[0], p2[0], 100)
    shift=[a*abs(x[i]-mid_x)**2 -a*(p1[0]-mid_x)**2 for i in range(100)]
    y =[p1[1]+(p2[1]-p1[1])*(x[i]-p1[0])/(p2[0]-p1[0]) +shift[i] for i in range(100)]
    return(x,y)

# x,y=curve_between((0,0),(1,0))
# fig=plt.figure()
# plt.plot(x,y)   
# plt.show()

## Map related display

In [10]:
class MapDisplay:
    def __init__(self,
                 station_location,
                 zoom,
                 travel_mode='driving-car',
                 time_unit='min',
                 distance_unit='km',
                 dash_array=[1, 10],
                 ant_speed=1000,
                 path_color='#FF0000',
                 pulse_color='#3f6fba',
                 path_weight=7
                ):
        # travel_mode: 'driving-car','foot-walking','cycling-regular', etc. Refer to ORS site for more options.
        # distance_unit: 'm','km'
        # time_unit: 'sec','min','h'
        self.station_location=station_location
        self.zoom=zoom
        self.travel_mode=travel_mode
        self.time_unit=time_unit
        self.distance_unit=distance_unit
        self.station_num=len(station_location)
        self.center=(sum([self.station_location[i][0] for i in range(self.station_num)])/self.station_num,
                     sum([self.station_location[i][1] for i in range(self.station_num)])/self.station_num)
        self.distance_matrix,self.duration_matrix=self.obtain_distance_duration_matrix(self.station_location,
                                        self.travel_mode,self.time_unit,self.distance_unit)
        self.dash_array=dash_array
        self.ant_speed=ant_speed
        self.path_color=path_color
        self.pulse_color=pulse_color
        self.path_weight=path_weight
        self.basemap=basemaps.OpenStreetMap.Mapnik
        self.m = Map(basemap=self.basemap, center=self.center, zoom=self.zoom)
        display(self.m)
        self.m.add_control(LayersControl())
        
        self.marker_l=[]
        marker_num =self.station_num
        for i in range(marker_num):
            self.marker_l.append(Marker(location=tuple(self.station_location[i]), draggable=False))
            self.m.add_layer(self.marker_l[i])
            self.marker_l[i].n_id=i  
        self.base_layer_group = LayerGroup(layers=tuple(self.marker_l))

    @staticmethod
    def obtain_distance_duration_matrix(location_list,travel_mode='driving-car',time_unit='min',distance_unit='km'):
        l_list=[[location_list[i][1],location_list[i][0]] for i in range(len(location_list))]
        body = {"locations":l_list,
           "metrics":['distance', 'duration'],'units':distance_unit}
        headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf624867668b6455a14e179c08e135929b526e',
        'Content-Type': 'application/json; charset=utf-8'
                }
        temp_str='https://api.openrouteservice.org/v2/matrix/%s' % travel_mode
        call = requests.post(temp_str, json=body, headers=headers)
        distance_matrix=call.json()['distances']
        if time_unit=='sec':
            duration_matrix=call.json()['durations']
        if time_unit=='min':
            temp=np.matrix(call.json()['durations'])/60
            duration_matrix= temp.tolist()
        if time_unit=='h':
            temp=np.matrix(call.json()['durations'])/3600
            duration_matrix= temp.tolist()
        return [distance_matrix,duration_matrix]
    
    @staticmethod
    def obtain_distance_duration_two_points(start_node,end_node,travel_mode='driving-car',time_unit='min',distance_unit='km'):
        location_list=[start_node,end_node]
        distance_matrix,duration_matrix=obtain_distance_duration_matrix(location_list,travel_mode,time_unit,distance_unit)
        distance=distance_matrix[0][1]
        duration=duration_matrix[0][1]
        return [distance,duration]
    
    @staticmethod
    def obtain_route(start_node,end_node,travel_mode='driving-car'):
        # eg. start_node=[40.719783, -74.003596],s
        # results doesn't include the starting and ending point
        headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
                }
        temp_str='https://api.openrouteservice.org/v2/directions/%s?api_key=5b3ce3597851110001cf624867668b6455a14e179c08e135929b526e&start=%s,%s&end=%s,%s' % (travel_mode,start_node[1],start_node[0],end_node[1],end_node[0])
        call = requests.get(temp_str, headers=headers)
        temp_list=call.json()['features'][0]['geometry']['coordinates']
        # notice the coordinates need to be flipped
        route_node_list=[[temp_list[i][1],temp_list[i][0]] for i in range(len(temp_list))]
        return route_node_list
    
    def show_route(self,start_node,end_node):
        # draw antpath between two nodes on the map
        route_node_list=self.obtain_route(start_node,end_node,self.travel_mode)
        ant_path = AntPath(
            locations=route_node_list,
            dash_array=self.dash_array,
            delay=self.ant_speed,
            color=self.path_color,
            pulse_color=self.pulse_color,
            weight=self.path_weight
                        )
        self.m.add_layer(ant_path)
    
    def take_sim_result(self,sim_result,max_soc):
        self.sim_result=sim_result
        self.max_soc=max_soc
    
    @staticmethod
    def station_click_handler(h,**kwargs):
        if kwargs.get('type') == 'click':
#             print('clicked',h)
            pass
        
    def show_station_status(self,t):
        self.m.clear_layers()
        self.m.add_layer(self.basemap)
        self.m.add_layer(self.base_layer_group)
        df=self.sim_result
        dft=df.loc[df['Time']==t]
        dfts=dft[dft['NodeID'].str.contains('t0')]
        z=[]
        for i in range(len(dfts)):
            s=np.zeros(self.max_soc)
            for j in range(self.max_soc):
                s[j]=dft['L%d' % j].values[i]
            z.append(int(sum(s)))
        message=[]
        for i in range(self.station_num):
            message.append(HTML())
            message[i].value = 'Available EV %s' % z[i]
            self.marker_l[i].popup = message[i]
        
        # on_click event
        for i in range(self.station_num):
            self.marker_l[i].on_click(partial(self.station_click_handler,i))
        


## Demand sampling

In [11]:
n_s = 8 # number of stations
n_tau = 10 # number of periods in the horizon
seed = 40
np.random.seed(seed)

# demand range per time step
d_low= 1
d_high= 2
# sample mean demand
d_mean={}
for i in range(n_s):
    for j in range(n_s):
        for k in range(n_tau):
            if i == j:
                d_mean[i, j, k] = 0
            else:
                d_mean[i, j, k] = np.random.uniform(d_low,d_high)

[demand_m,d_period,d_ori,d_des] =sp.demand_s(n_s,n_tau,d_mean,seed)
# print(demand_m)

# constant demand
demand_test={}
for i in range(n_s):
    for j in range(n_s):
        for k in range(n_tau):
            if i == j:
                demand_test[i, j, k] = 0
            else:
                demand_test[i, j, k] = 1
# print(demand_test)

def store_demand(n_s,n_tau,demand_dic):
    with open('./Data/demand_file.csv', 'w', newline='') as csvfile:
        fieldnames = ['origin', 'destination','period','amount']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(n_s):
            for j in range(n_s):
                for k in range(n_tau):
                    writer.writerow({'origin': '%d'%i, 'destination': '%d'%j, 'period': '%d'%k, 'amount':'%d'%int(demand_dic[i,j,k])})

# store_demand(n_s,n_tau,demand_test)

## Sample test

In [12]:
## test 1
# sample_map=[(0,0),(1,0)]
# sample_distance={}
# sample_distance[0,1]=2
# sample_distance[1,0]=3
# nw=Network(sample_map,sample_distance,7,10,2,1,25,500,demand_test,[0.9,0.2])
# nw.deploy_vehicle([10,20])

## test 2
# sample_map=[(0,0),(1,0),(1,1)]
# sample_distance={}
# sample_distance[0,1]=2
# sample_distance[1,0]=3
# sample_distance[0,2]=1
# sample_distance[2,0]=2
# sample_distance[1,2]=3
# sample_distance[2,1]=2
# nw=Network(sample_map,sample_distance,7,10,2,1,25,500,demand_test,[0.9,0.2])
# nw.deploy_vehicle([10,20,15])
# # print(nw.stations)
# nw.run()
# nw.show_data()

## Show animation

In [13]:
# @interact(t=widgets.IntSlider(min=0,max=9,step=1,value=0))
# def demo(t):
# #     nw.show_frame(t)
#     nw.show_route(t,1,0)
#     nw.show_node(t,nw.stations[1])
#     nw.show_map(t)
#     nw.show_node_histogram(t,nw.stations[1])


## Incorporate the map

In [14]:
# station_location=[[40.762646, -73.988237],[40.746002, -74.006090],[40.728486, -74.007509],[40.710488, -74.016218],
#          [40.719075, -73.999738],[40.710488, -73.994588],[40.715562, -73.984460]]
# md=MapDisplay(station_location,13)
# segment_len=3 # length of each road segment
# distance_2={}
# for i in range(md.station_num):
#     for j in range(md.station_num):
#         distance_2[i,j]=math.ceil(md.distance_matrix[i][j]/segment_len)
# # print(distance_2)
# nw_2=Network(station_location,distance_2,7,10,2,1,25,500,demand_test,[0.9,0.2])
# nw_2.deploy_vehicle([10,20,15,10,15,15,20])
# nw_2.run()
# md.take_sim_result(nw_2.sim_result,nw_2.max_soc)
# # print(md.sim_result)


In [15]:
# @interact(t=widgets.IntSlider(min=0,max=9,step=1,value=0))
# def map_demo(t):
#     md.show_station_status(t)

In [16]:
# md.show_route([40.719783, -74.003596],[40.759783, -74.053596])

In [17]:
# add plot methods in the MapDisplay class
# 1 click to show histogram plot
# check out the web app settings

In [18]:
# print(md.distance_matrix)
# md.show_route([40.762646, -73.988237],[40.746002, -74.006090])
# md.m.clear_layers()
# bl=basemaps.OpenStreetMap.Mapnik
# md.m.add_layer(bl)
# print(md.marker_l[0].n_id,md.marker_l[1].n_id)